In [42]:
# pull in dependencies needed from sportsreference api
from sportsreference.mlb.teams import Teams
from sportsreference.mlb.teams import Roster
from sportsreference.mlb.roster import Player

# pull in all other dependencies needed
import itertools
import pandas as pd

In [2]:
# to create a list of player ids:
# set up an empty array that we will append to
player_list = []

# 1) create a loop to go through years
for year in range(1869,2020):

# 2) create a loop to go through teams and pull out the player names and their ids
    for team in Teams(year):
        player_data = Roster(team.abbreviation, year = year, slim=True).players
        player_list.append(player_data)

In [3]:
# convert the dictionay of arrays to an array of arrays using list comprehension
info = [list(x.keys()) for x in player_list]

In [4]:
# flatten the list using itertools
merged = list(itertools.chain(*info))

In [5]:
# create a pandas data frame that will show all of the player ids
df = pd.DataFrame(merged)

In [6]:
# use .unique() on the df to clear out any duplicate players
unique_player_id = df[0].unique()

In [7]:
# to create a list of data frames for each player's stats:
# set up an empty array that we will append to
player_df_list = []

# create a loop that will find each players' id and create a data frame of relevant statistics
for index, player_id in enumerate(unique_player_id):
    player_df = Player(player_id).dataframe[["player_id", "name", "at_bats", "runs", "hits", "runs_batted_in", "bases_on_balls", "batting_average", "doubles", "triples", "home_runs", "stolen_bases"]]
    player_df_list.append(player_df)
    print(index, len(unique_player_id))

0 1410
1 1410
2 1410
3 1410
4 1410
5 1410
6 1410
7 1410
8 1410
9 1410
10 1410
11 1410
12 1410
13 1410
14 1410
15 1410
16 1410
17 1410
18 1410
19 1410
20 1410
21 1410
22 1410
23 1410
24 1410
25 1410
26 1410
27 1410
28 1410
29 1410
30 1410
31 1410
32 1410
33 1410
34 1410
35 1410
36 1410
37 1410
38 1410
39 1410
40 1410
41 1410
42 1410
43 1410
44 1410
45 1410
46 1410
47 1410
48 1410
49 1410
50 1410
51 1410
52 1410
53 1410
54 1410
55 1410
56 1410
57 1410
58 1410
59 1410
60 1410
61 1410
62 1410
63 1410
64 1410
65 1410
66 1410
67 1410
68 1410
69 1410
70 1410
71 1410
72 1410
73 1410
74 1410
75 1410
76 1410
77 1410
78 1410
79 1410
80 1410
81 1410
82 1410
83 1410
84 1410
85 1410
86 1410
87 1410
88 1410
89 1410
90 1410
91 1410
92 1410
93 1410
94 1410
95 1410
96 1410
97 1410
98 1410
99 1410
100 1410
101 1410
102 1410
103 1410
104 1410
105 1410
106 1410
107 1410
108 1410
109 1410
110 1410
111 1410
112 1410
113 1410
114 1410
115 1410
116 1410
117 1410
118 1410
119 1410
120 1410
121 1410
122 1410
123

In [8]:
# Use .loc function in pandas to just obtain the career stats for every player
career_df_list = []
for df in player_df_list:
    career_df_list.append(df.loc["Career"])

In [34]:
# use concat to re-write the list of data frames as a single data frame
clean_df = pd.concat(career_df_list)

# and drop the duplicates of names using either 'player_id' or 'name'
mlb_history_df = clean_df.drop_duplicates(subset=['player_id'])
mlb_history_df.head()

,player_id,name,at_bats,runs,hits,runs_batted_in,bases_on_balls,batting_average,doubles,triples,home_runs,stolen_bases
Career,chiriro01,Robinson Chirinos,1832,252.0,423.0,269,203.0,0.231,100,4,86.0,4.0
Career,gourryu01,Yuli Gurriel,1971,264.0,565.0,301,99.0,0.287,135,5,71.0,14.0
Career,altuvjo01,José Altuve,5177,766.0,1610.0,556,377.0,0.311,308,28,133.0,256.0
Career,correca01,Carlos Correa,2269,334.0,626.0,397,272.0,0.276,128,7,107.0,33.0
Career,bregmal01,Alex Bregman,2058,365.0,582.0,342,309.0,0.283,152,12,105.0,34.0


In [32]:
# read in the hall of fame csv
hall_of_fame_df = pd.read_csv('Resources/Hall_of_Fame_List.csv')
hall_of_fame_df.head()

,Year,Name,Primary position,Primary team,Career,Induction method,Vote %
0,1936.0,Ty Cobb,CF,Detroit Tigers,1905–1928,BBWAA,98.23%
1,1936.0,Walter Johnson,P,Washington Senators,1907–1927,BBWAA,83.63%
2,1936.0,Christy Mathewson,P,New York Giants,1900–1916,BBWAA,90.71%
3,1936.0,Babe Ruth,RF,New York Yankees,1914–1935,BBWAA,95.13%
4,1936.0,Honus Wagner,SS,Pittsburgh Pirates,1897–1917,BBWAA,95.13%


In [37]:
# clean the csv file to show only the names of the hall of famers and status
hall_of_fame_df = hall_of_fame_df.dropna()
hof_df = hall_of_fame_df.rename(columns={"Name":"name", "Primary position": "position"})
hof_cleaned = hof_df[["name", "position"]]
hof_cleaned.head()

,name,position
0,Ty Cobb,CF
1,Walter Johnson,P
2,Christy Mathewson,P
3,Babe Ruth,RF
4,Honus Wagner,SS


In [41]:
# join the no_duplicates_df with hof_cleaned to show career stats and hall of fame status
merge_df = mlb_history_df.merge(hof_cleaned, on='name', how='outer')
final_df = merge_df.fillna('no').rename(columns={"position": "hall_of_fame"})
final_df

,player_id,name,at_bats,runs,hits,runs_batted_in,bases_on_balls,batting_average,doubles,triples,home_runs,stolen_bases,hall_of_fame
0,chiriro01,Robinson Chirinos,1832,252,423,269,203,0.231,100,4,86,4,no
1,gourryu01,Yuli Gurriel,1971,264,565,301,99,0.287,135,5,71,14,no
2,altuvjo01,José Altuve,5177,766,1610,556,377,0.311,308,28,133,256,no
3,correca01,Carlos Correa,2269,334,626,397,272,0.276,128,7,107,33,no
4,bregmal01,Alex Bregman,2058,365,582,342,309,0.283,152,12,105,34,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1737,no,Lee Smith,no,no,no,no,no,no,no,no,no,no,P
1738,no,Derek Jeter,no,no,no,no,no,no,no,no,no,no,SS
1739,no,Larry Walker,no,no,no,no,no,no,no,no,no,no,RF
1740,no,Marvin Miller,no,no,no,no,no,no,no,no,no,no,EXEC


In [43]:
# export as a csv to Resources folder
final_df.to_csv('Resources/player_data.csv', index=False)